#### 1) Попытаться ответить на вопросы/выдвинуть гипотезы

- А точно нужно сортировать по вероятности?

У нас, скорее,задача выдачи топ-5, чем ранжирования, т.к. 5 наименований - это довольно мало. Если речь все-таки идет о сортировке, возможно, у менеджера есть свои приоритеты: например, поставить товар для роста среднего чека в самом начале списка. 

- Какую метрику использовать?

Precision@k. Возможно, что-то вроде money MAP @k для оценки результата по всем юзерам c оглядкой на деньги, т.к. важен рост среднего чека. 

- Сколько раз в неделю отпрпавляем рассылку?

Зависит от того, сколько раз в неделю обновляются акции и т.п. Вероятно, рассылать почти одно и то же каждый день нет смысла, а также ведет к лишним затратам. Возможно, нужны какие-то АБ-тесты, чтобы выбрать оптимальное кол-во рассылок

- В какое время отправляем рассылку?

После появления в системе информации о новых акциях. Если у нас есть данные, в какое время пользователи обычно заходят на сайт, то можно подстраиваться под это время (например, больше всего пользователей у нас в субботу утром, рассылку можно делать в пятницу вечером)

- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?

Разные акции, фильтровать и удалять товары, которые пользователь видел в прошлый раз или уже купил, добавлять сезонные и праздничные товары

- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?

Разные - товары, между которыми большое расстояние в itemitem матрице. С другой стороны, можно договориться о том, что именно менеджер считает "разными" товарами и насколько "разными" они должны быть. Возможно, имеет смысл тем пользователям, которые уже покупают у нас разные товары, предлагать разные, а тем пользователям, которые покупают товары из одной категории, предлагать что-то похожее (+, возможно, один необычный товар на пробу). Можно опробовать разные способы и скорректировать их в зависимости от дальнейшего поведения пользователей. 

- **3 товара с акцией** (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)

Если формировать user-item матрицу как на вебинаре (складывать кол-во купленного товара за предыдущие недели), то изменение цены за предыдущие недели не будет учитываться, только общее кол-во. Если нужно, можно включить в матрицу только товары со скидкой (в столбце retail-disc не ноль), можно считать товары с разной скидкой разными товарами (разделить их item_id), но, видимо, это плохой способ, т.к. матрица разрастется. Кажется, можно было бы составить модель без учета скидок за предыдущие периоды, договориться с менеджером, вклчать ли в рекоммендации товары, которые пользователь уже покупал, но давно (например, 2 месяца назад купил, а сейчас на товар появилась акция), получить от коллег список товаров, на которые планируется следующая акция, и выдать рекоммендацию на топ-3 товара только из этого списка.

- **1 новый товар** (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?) 

Может быть, предложить товар, который купили похожие пользователи, а этот еще не купил? Или хотя бы популярный товар

- **1 товар для роста среднего чека** (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)

Смотрим цены товаров, которые покупал пользователь. При выдаче рекомендации оставляем 1 товар, цена которого выше 0,75% квантиля из списка уже купленных. 

#### 4) Поэкспериментировать с ALS (grid-search)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from metrics import precision_at_k, recall_at_k

In [2]:
data = pd.read_csv('C:/Users/yana/3/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [5]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [13]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999_999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix.head(3)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [17]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)]
    return res

In [18]:
user_item_matrix = bm25_weight(user_item_matrix.T).T

In [19]:
params = {
    'factors': [48, 64, 96, 128], 
    'regularization': [0.03, 0.05, 0.1],
    'iterations': [10, 15, 20], 
    'num_threads': [5, 10, 15]
}
table = pd.DataFrame(columns = ['factors','regularization', 'iterations', 'num_threads' ,'precision@k'])

for i in params['factors']:
    for j in params['regularization']:
        for k in params['iterations']:
            for l in params['num_threads']:
                model = AlternatingLeastSquares(factors=i, 
                                regularization=j,
                                iterations=k, 
                                num_threads=l,
                                calculate_training_loss=True)

                model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=False)
                result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
                table.loc[len(table)] = [i, j, k, l, result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()]

In [20]:
#лучшее сочетание параметров
table.sort_values('precision@k', ascending=False).head(1)

,factors,regularization,iterations,num_threads,precision@k
92,128.0,0.05,10.0,15.0,0.230069
